In [6]:
import sqlite3
import pandas as pd
import datetime

conn = sqlite3.connect('../database.sqlite')
query_matches = \
    'select match_api_id, home_team_api_id, away_team_api_id, home_team_goal, away_team_goal, date, \
    `B365H`, `B365A`, `B365D` \
    from Match;'
matches = pd.read_sql(query_matches, conn, index_col='match_api_id', parse_dates=['date']);
matches.head(6)

,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,date,B365H,B365A,B365D
match_api_id,,,,,,,,
492473,9987,9993,1,1,2008-08-17,1.73,5.00,3.40
492474,10000,9994,0,0,2008-08-16,1.95,3.60,3.20
492475,9984,8635,0,3,2008-08-16,2.38,2.75,3.30
492476,9991,9998,5,0,2008-08-17,1.44,7.50,3.75
492477,7947,9985,1,3,2008-08-16,5.00,1.65,3.50
492478,8203,8342,1,1,2008-09-24,4.75,1.67,3.40


In [7]:
import pandas as pd
def teamattrs(team_api_id, matchdate):
    """
    Helper method that gets the specified team's attributes from
    the date closest to matchdate.
    
    Keyword arguments:
    team_api_id -- the api_id of the team of interest (Integer)
    matchdate -- pd.Timestamp
    """
    query = \
        'select date, buildUpPlaySpeed, buildUpPlayPassing, \
                chanceCreationPassing, chanceCreationCrossing, chanceCreationShooting, \
                defencePressure, defenceAggression, defenceTeamWidth \
        from Team_Attributes \
        where team_api_id = %s \
        order by date asc;' % team_api_id
    attrs = pd.read_sql(query, conn, parse_dates=['date'])
    if (not attrs.empty):
        closestattrs = None
        mindiff = None
        for index, row in attrs.iterrows():
            date = row['date']
            diff = abs(matchdate - date)
            if (mindiff is None or diff < mindiff):
                mindiff = diff
                closestattrs = row
        return closestattrs.drop('date')
    else:
        return None
    
print('Example output from teamattrs:\n')
teamattrs(9993, pd.to_datetime('2008-08-16'))

Example output from teamattrs:



buildUpPlaySpeed          35
buildUpPlayPassing        35
chanceCreationPassing     45
chanceCreationCrossing    40
chanceCreationShooting    50
defencePressure           70
defenceAggression         70
defenceTeamWidth          70
Name: 0, dtype: object

In [9]:
def differencevector(row):
    """
    This function is applied to each row in the 'matches' DataFrame.
    For each row, it returns a pandas.Series containing the number of
    goals scored by each team and the result from the home team's
    perspective.
    
    Keyword arguments:
    row -- pandas.Series
    """
    date = row['date']
    homegoal = row['home_team_goal']
    homeattrs = teamattrs(row['home_team_api_id'], date)
    
    awaygoal = row['away_team_goal']
    awayattrs = teamattrs(row['away_team_api_id'], date)
    
    if (homeattrs is not None and awayattrs is not None):
        differencevector = homeattrs - awayattrs

        if (homegoal > awaygoal):
            result = 2
        elif (awaygoal == homegoal):
            result = 1
        else:
            result = 0

        results = pd.Series({
            'home_team_odds': row['B365H'],
            'away_team_odds': row['B365A'],
            'draw_odds': row['B365D'],
            'outcome':  result
        })
        return results.append([differencevector])
    else:
        return

results = matches.apply(differencevector, axis=1).dropna(axis=0, how='any')
conn.close()

results.to_csv('../difference_vectors.csv')

,away_team_api_id,home_team_api_id,outcome,buildUpPlaySpeed,buildUpPlayPassing,chanceCreationPassing,chanceCreationCrossing,chanceCreationShooting,defencePressure,defenceAggression,defenceTeamWidth
match_api_id,,,,,,,,,,,
492473,9993.0,9987.0,1.0,10.0,10.0,5.0,-5.0,10.0,0.0,-5.0,0.0
492474,9994.0,10000.0,1.0,5.0,0.0,10.0,0.0,0.0,5.0,-5.0,0.0
492475,8635.0,9984.0,0.0,-5.0,0.0,0.0,-5.0,-5.0,-5.0,10.0,0.0
492476,9998.0,9991.0,2.0,20.0,15.0,5.0,2.0,14.0,-1.0,5.0,-7.0


In [10]:
results.head()

,away_team_odds,draw_odds,home_team_odds,outcome,buildUpPlaySpeed,buildUpPlayPassing,chanceCreationPassing,chanceCreationCrossing,chanceCreationShooting,defencePressure,defenceAggression,defenceTeamWidth
match_api_id,,,,,,,,,,,,
492473,5.00,3.40,1.73,1.0,10.0,10.0,5.0,-5.0,10.0,0.0,-5.0,0.0
492474,3.60,3.20,1.95,1.0,5.0,0.0,10.0,0.0,0.0,5.0,-5.0,0.0
492475,2.75,3.30,2.38,0.0,-5.0,0.0,0.0,-5.0,-5.0,-5.0,10.0,0.0
492476,7.50,3.75,1.44,2.0,20.0,15.0,5.0,2.0,14.0,-1.0,5.0,-7.0
492478,1.67,3.40,4.75,1.0,30.0,20.0,5.0,-10.0,5.0,0.0,0.0,-10.0
